In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Load the data
df = pd.read_csv('train.csv')

# Extract the relevant columns
text_columns = [
    'Girisimcilikle Ilgili Deneyiminiz Var Mi?',
    'Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?'
]
target_column = 'Degerlendirme Puani'

# Drop rows with missing target values
df = df.dropna(subset=[target_column])

# Fill missing values in text columns with empty strings
df[text_columns] = df[text_columns].fillna('')

# Combine text columns into one for processing
df['combined_text'] = df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Tokenize and preprocess the text data for Word2Vec
def preprocess_text(text):
    return simple_preprocess(text, deacc=True)

# Prepare the Word2Vec model
sentences = [preprocess_text(text) for text in df['combined_text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create document vectors by averaging word vectors
def vectorize_text(text):
    tokens = preprocess_text(text)
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if len(vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

df['text_vector'] = df['combined_text'].apply(vectorize_text)
word2vec_features = np.array(df['text_vector'].tolist())

# TF-IDF Vectorization
text_data = df['combined_text']
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_vectors = tfidf_vectorizer.fit_transform(text_data)

# Combine Word2Vec and TF-IDF features
combined_features = hstack([tfidf_vectors, word2vec_features])

# Target variable
y = df[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Print actual vs predicted values
print("\nSample Actual vs. Predicted Values:")
for actual, predicted in zip(y_test.head(10), y_pred[:10]):
    print(f"Actual: {actual}, Predicted: {predicted}")

# Optionally, save the model for future use
import joblib
joblib.dump(model, 'combined_regression_model.pkl')
joblib.dump(scaler, 'combined_scaler.pkl')


/var/folders/0p/hymvzzjx3cb75ylj_41b1d8h0000gn/T/ipykernel_80508/2610565008.py:13: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


Mean Squared Error: 284.30

Sample Actual vs. Predicted Values:
Actual: 36.0, Predicted: 38.28853832300412
Actual: 68.0, Predicted: 47.80804211174352
Actual: 7.0, Predicted: 28.272426864511054
Actual: 30.0, Predicted: 42.575595332689936
Actual: 11.0, Predicted: 28.272426864511054
Actual: 46.0, Predicted: 27.157834694483533
Actual: 31.0, Predicted: 28.272426864511054
Actual: 21.0, Predicted: 44.002439195740344
Actual: 15.0, Predicted: 28.272426864511054
Actual: 15.0, Predicted: 27.157834694483533


['combined_scaler.pkl']

In [3]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

# Define a pipeline to simplify the process
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('scaler', StandardScaler(with_mean=False)),
    ('regressor', Ridge())
])

# Parameter grid for GridSearchCV
param_grid = {
    'tfidf__max_features': [500, 1000, 1500],
    'regressor__alpha': [0.1, 1, 10]
}

# Grid search for best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(text_data, target)

# Best model and parameters
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Evaluate on test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')



NameError: name 'target' is not defined